### Import libraries and functions

In [1]:
from vis_utils import load_volume, VolumeVisualizer, ColorMapVisualizer, draw_graph_on_model
from identification import get_data_graph, generate_sphere, clean_data_graph, parametrize_graph, get_vessel_graph, remove_graph_components, connect_endings_3d, get_starting_points,sphere_full, get_points_of_interest

In [17]:
import pickle
import numpy as np
# import matplotlib.pyplot as plt
from skimage import measure
from scipy.ndimage import zoom
from skimage.morphology import ball
# from scipy.signal import fftconvolve
from scipy.ndimage import distance_transform_edt
# from skimage.morphology import skeletonize_3d, binary_dilation, convex_hull_image, ball, area_closing

### Flags

In [18]:
visualize_steps = True 

### Utils

In [19]:
def visualize_mask_bin(mask):
    VolumeVisualizer((mask > 0).astype(np.uint8), binary=True).visualize()
    
def visualize_mask_non_bin(mask):
    VolumeVisualizer((mask > 0).astype(np.uint8) * 255, binary=False).visualize()

### Read data

In [20]:
volume = np.fromfile('../data/P13/data.raw', dtype=np.uint8)
volume = volume.reshape(877, 488, 1132)

### Cut image

In [21]:
volume = volume[200:700, :, 100:650]
val = 20

volume[:, 200:, -35:] = val
volume[-150:, 200:, -100:-35] = val
volume[:, -200:-150, -100:-35] = val
volume[:, :210, -50:] = val
volume[:150, :150, -85:-50] = val

volume = volume[-170:-40,200:480,30:260]
volume[-90:, :, -110:] = val
volume[:, -80:, -92:] = val

In [22]:
volume.shape

(130, 280, 230)

### Segmentation

In [23]:
mask = volume > 31

### Vizualization

In [9]:
if visualize_steps:
    VolumeVisualizer(mask, binary=True).visualize() 

In [24]:
scaled_mask = zoom(mask, zoom=0.7, order=0)

In [25]:
scaled_mask.shape

(91, 196, 161)

### EDT transform

In [26]:
%%time
edt_img = distance_transform_edt(scaled_mask)

CPU times: total: 469 ms
Wall time: 465 ms


In [27]:
if visualize_steps:
    ColorMapVisualizer(edt_img.astype(np.uint8)).visualize()

### Graph creation

In [28]:
import numpy as np
import networkx as nx

from scipy.ndimage import distance_transform_edt
from scipy.spatial.distance import euclidean
from skimage.draw import circle_perimeter, disk
from skimage.feature import peak_local_max

def get_points_of_interest(point, radius, visited, edt_img):
    """Get list of points that can be on the vessel centerline. The method utilizes local maxima
    to find the centerline on the basis of the EDT image of the vessel. The returned points are
    sorted by the EDT image value, with the pixels on the largest vessel at the beginning of the
    list.

    Args:
        point (tuple(int, int)): Point on centerline.
        radius (int): Radius of search circle.
        visited (np.array(bool)): Boolean array of visited image pixels.

    Returns:
        list: A list of points that can be on the vessel centerline.
    """
    # get points on circle circumference
    #circle_points = circle_perimeter(*point, radius=radius, method="andres")
    circle_points = get_points_on_sphere(point, radius)
    circle_points = list(filter(lambda x: is_inside_image(x, edt_img.shape), circle_points))

    #print("Circle points: ", len(circle_points), edt_img.shape)
    # create image with edt values of point on circle circumference
    helper = np.zeros_like(edt_img)
    
    for p in circle_points:
        helper[p] = edt_img[p]
    # find local maxima
    coords = peak_local_max(
        helper, min_distance=radius // 3, threshold_abs=2, threshold_rel=0.5
    )
    #print("Local peaks on surface: ", len(coords))
    # find coordinates of local 
    
    for p in circle_points:
        helper[p] = 0
    helper[tuple(coords.T)] = 1
    helper[visited] = 0
    points_of_interest = list(zip(*np.nonzero(helper)))
    
    points_of_interest = list(filter(lambda x: is_inside_image(x, edt_img.shape), points_of_interest))
    # sort points by vessel diameter
    points_of_interest.sort(key=lambda x: edt_img[x], reverse=True)
    return points_of_interest


def get_points_on_sphere(center, r):
    b = generate_sphere(r)
    non_zero = np.array(tuple(zip(*np.nonzero(b))))
    new_points = []

    for point in non_zero:
        x,y,z = point[0], point[1], point[2]
        xc, yc, zc = center

        x,y,z = x-r, y-r, z-r
        x,y,z = x + xc, y + yc, z + zc
        new_points.append((x,y,z))
    return new_points


def is_inside_image(p, shape):
    return p[0] >= 0 and p[1] >= 0 and p[2] >= 0 and p[0] < shape[0] and p[1] < shape[1] and p[2] < shape[2]

def generate_sphere(radius):
    large = ball(radius)
    small = ball(np.maximum(1, radius-1))
    padded_small = np.pad(small, 1, mode='constant')
    sphere = large - padded_small
    return sphere

def get_vessel_graph(seg_img):
    """Produce a graph which describes the vessel structure.

    Args:
        seg_img (np.array): Segmentation of vessel image.

    Returns:
        nx.Graph: Directed graph describing the vessel structure.
    """
    # perform euclidean distance transform on base image
    edt_img = distance_transform_edt(seg_img)
    # get starting points
    starting_point_stack = get_starting_points(edt_img)
    # create vessel tree data structure
    vessel_graph = nx.DiGraph()
    # create array for holding information on visited pixels
    visited = np.zeros_like(edt_img).astype(bool)
    
    i = 0
    print("Starting points: ", len(starting_point_stack))
    while starting_point_stack:
        # select new starting point and check if it was not visited already
        starting_point = starting_point_stack.pop()
        if visited[starting_point]:
            continue

        vessel_graph.add_node(starting_point)
        points_to_examine = [starting_point]
        
        j = 0
        while points_to_examine:
            if j % 10 == 0:
                print(f"i: {i} j: {j} {len(points_to_examine)}")
            # get point from queue
            point = points_to_examine.pop()
            # calculate vessel radius
            radius = int(edt_img[point])
            # if radius is too small, don't go further into this vessel
            # also limited by min_distance having to be >= 1 in peak_local_max
            if radius <= 2:
                continue
            # get points that can be on centreline
            points_of_interest = get_points_of_interest(point, radius, visited, edt_img)

            # eliminate points that are too close to each other
            for poi_1 in points_of_interest:
                for poi_2 in points_of_interest:
                    if poi_1 != poi_2 and euclidean(poi_1, poi_2) < edt_img[poi_1]:
                        points_of_interest.pop(points_of_interest.index(poi_2))
            # add points of interest to examination list and to vessel graph
            for poi in points_of_interest:
                # avoid node duplication
                if not (poi in points_to_examine or vessel_graph.has_node(poi)):
                    points_to_examine.append(poi)
                    vessel_graph.add_edge(point, poi)
            # remove potential centreline pixels next to analyzed point to prevent going backwards
            disk_points = sphere_full(point, radius, visited.shape)
            visited[disk_points] = True
            j += 1
        i += 1

    return vessel_graph.to_undirected()

In [29]:
%%time
vessel_graph = get_vessel_graph(scaled_mask)

Starting points:  4286
i: 0 j: 0 1
i: 0 j: 10 8
i: 0 j: 10 7
i: 0 j: 20 6
i: 0 j: 30 2
i: 0 j: 40 4
i: 1 j: 0 1
i: 2 j: 0 1
i: 2 j: 10 3
i: 3 j: 0 1
i: 3 j: 10 2
i: 3 j: 10 1
i: 4 j: 0 1
i: 4 j: 10 2
i: 4 j: 10 1
i: 5 j: 0 1
i: 6 j: 0 1
i: 7 j: 0 1
i: 8 j: 0 1
i: 9 j: 0 1
i: 10 j: 0 1
i: 11 j: 0 1
i: 12 j: 0 1
i: 12 j: 10 3
i: 12 j: 20 1
i: 13 j: 0 1
i: 14 j: 0 1
i: 15 j: 0 1
i: 16 j: 0 1
i: 17 j: 0 1
i: 18 j: 0 1
i: 19 j: 0 1
i: 20 j: 0 1
i: 21 j: 0 1
i: 22 j: 0 1
i: 23 j: 0 1
i: 24 j: 0 1
i: 25 j: 0 1
i: 26 j: 0 1
i: 27 j: 0 1
i: 28 j: 0 1
i: 29 j: 0 1
i: 30 j: 0 1
i: 31 j: 0 1
i: 32 j: 0 1
i: 33 j: 0 1
i: 34 j: 0 1
i: 35 j: 0 1
i: 36 j: 0 1
i: 37 j: 0 1
i: 38 j: 0 1
i: 39 j: 0 1
i: 40 j: 0 1
i: 41 j: 0 1
i: 42 j: 0 1
i: 43 j: 0 1
i: 44 j: 0 1
i: 45 j: 0 1
i: 46 j: 0 1
i: 47 j: 0 1
i: 48 j: 0 1
i: 49 j: 0 1
i: 50 j: 0 1
i: 51 j: 0 1
i: 52 j: 0 1
i: 53 j: 0 1
i: 54 j: 0 1
i: 55 j: 0 1
i: 56 j: 0 1
i: 57 j: 0 1
i: 58 j: 0 1
i: 59 j: 0 1
i: 60 j: 0 1
i: 61 j: 0 1
i: 62 j: 0 1
i: 63 j: 0

In [30]:
print('Number of nodes', len(vessel_graph.nodes))
print('Number of edges', len(vessel_graph.edges))
print('Average degree', sum(dict(vessel_graph.degree).values()) / len(vessel_graph.nodes))

Number of nodes 4279
Number of edges 359
Average degree 0.16779621406870765


In [31]:
if visualize_steps:
    model_with_graph = draw_graph_on_model(scaled_mask, vessel_graph)
    ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

In [ ]:
pickle.dump(vessel_graph3, open('only_vertices_simple_get_vessel.pickle', 'wb'))

### Remove components

In [32]:
%%time
vessel_graph_rm = remove_graph_components(vessel_graph)

CPU times: total: 15.6 ms
Wall time: 13.8 ms


In [33]:
# print(nx.info(vessel_graph_rm))

print('Number of nodes', len(vessel_graph_rm.nodes))
print('Number of edges', len(vessel_graph_rm.edges))
print('Average degree', sum(dict(vessel_graph_rm.degree).values()) / len(vessel_graph_rm.nodes))

Number of nodes 394
Number of edges 359
Average degree 1.8223350253807107


In [ ]:
if visualize_steps:
    model_with_graph = draw_graph_on_model(scaled_mask, vessel_graph_rm)
    ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

In [ ]:
pickle.dump(vessel_graph_rm, open('removed-small.pickle', 'wb'))

### Connect endings 3D

In [34]:
%%time
vessel_graph_cn = connect_endings_3d(vessel_graph, edt_img)

Endings:  193
Endings:  192
Endings:  191
Endings:  190
Endings:  189
Endings:  188
Endings:  186
Endings:  185
Endings:  183
Endings:  181
Endings:  179
Endings:  178
Endings:  177
Endings:  176
Endings:  175
Endings:  174
Endings:  173
Endings:  171
Endings:  170
Endings:  169
Endings:  168
Endings:  166
Endings:  164
Endings:  162
Endings:  160
Endings:  159
Endings:  157
Endings:  156
Endings:  154
Endings:  152
Endings:  150
Endings:  148
Endings:  146
Endings:  144
Endings:  143
Endings:  142
Endings:  140
Endings:  139
Endings:  138
Endings:  137
Endings:  135
Endings:  134
Endings:  133
Endings:  132
Endings:  131
Endings:  130
Endings:  128
Endings:  127
Endings:  126
Endings:  125
Endings:  123
Endings:  122
Endings:  121
Endings:  120
Endings:  119
Endings:  118
Endings:  116
Endings:  114
Endings:  113
Endings:  112
Endings:  111
Endings:  110
Endings:  109
Endings:  108
Endings:  107
Endings:  106
Endings:  105
Endings:  104
Endings:  103
Endings:  102
Endings:  101
Ending

In [35]:
# print(nx.info(vessel_graph_cn))

print('Number of nodes', len(vessel_graph_cn.nodes))
print('Number of edges', len(vessel_graph_cn.edges))
print('Average degree', sum(dict(vessel_graph_cn.degree).values()) / len(vessel_graph_cn.nodes))

Number of nodes 395
Number of edges 389
Average degree 1.969620253164557


In [36]:
model_with_graph = draw_graph_on_model(scaled_mask, vessel_graph_cn)
ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

In [38]:
%%time
data_graph = parametrize_graph(vessel_graph_cn, edt_img)

CPU times: total: 31.2 ms
Wall time: 33 ms


C:\Users\Alicja\Alicja\medyczna\RenalVasculatureReconstruction\sphere_method\identification.py:425: RuntimeWarning: invalid value encountered in divide
  v1 /= np.linalg.norm(v1)
C:\Users\Alicja\Alicja\medyczna\RenalVasculatureReconstruction\sphere_method\identification.py:427: RuntimeWarning: invalid value encountered in divide
  v2 /= np.linalg.norm(v2)


In [39]:
%%time
data_graph_cl = clean_data_graph(data_graph)

CPU times: total: 0 ns
Wall time: 994 µs


In [40]:
model_with_graph = draw_graph_on_model(scaled_mask, data_graph_cl)
ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

In [ ]:
nx.get_edge_attributes(data_graph, "average_vessel_diameter").items(),